In [2]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error

In [3]:
class SegRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, segment_length):
        super(SegRNN, self).__init__()
        self.segment_length = segment_length
        self.gru_segment = nn.GRU(input_size, hidden_size, batch_first=True)
        self.gru_aggregate = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        num_segments = seq_len // self.segment_length
        x = x[:, :num_segments * self.segment_length, :]
        x = x.view(batch_size * num_segments, self.segment_length, -1)
        
        # Process each segment independently
        _, h_n_segment = self.gru_segment(x)
        h_n_segment = h_n_segment.squeeze(0).view(batch_size, num_segments, -1)
        
        # Aggregate segment embeddings
        _, h_n_aggregate = self.gru_aggregate(h_n_segment)
        h_n_aggregate = h_n_aggregate.squeeze(0)
        
        # Final output
        out = self.fc(h_n_aggregate)
        return out

In [4]:
class SegRNNModel(pl.LightningModule):
    def __init__(self, input_size, hidden_size, output_size, segment_length, learning_rate=0.001):
        super(SegRNNModel, self).__init__()
        self.model = SegRNN(input_size, hidden_size, output_size, segment_length)
        self.criterion = nn.MSELoss()
        self.learning_rate = learning_rate

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self(inputs)
        loss = self.criterion(outputs, targets)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self(inputs)
        loss = self.criterion(outputs, targets)
        self.log("val_loss", loss)
        return loss

    def test_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self(inputs)
        loss = self.criterion(outputs, targets)
        self.log("test_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

In [5]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import pytorch_lightning as pl

# Parameters for sanity check
input_size = 5      # Number of input features
hidden_size = 10    # Hidden size of the GRU
output_size = 3     # Number of output features
segment_length = 4  # Segment length for the SegRNN
sequence_length = 16  # Total sequence length (must be divisible by segment_length)
batch_size = 2      # Batch size
num_samples = 20    # Total samples in synthetic dataset

# Generate synthetic data
torch.manual_seed(42)
X = torch.randn(num_samples, sequence_length, input_size)  # Input: [num_samples, seq_length, input_size]
y = torch.randn(num_samples, output_size)                 # Target: [num_samples, output_size]

# Create DataLoader
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=batch_size)

# Initialize model
model = SegRNNModel(input_size=input_size, hidden_size=hidden_size, output_size=output_size, segment_length=segment_length)

# Lightning Trainer for quick sanity check
trainer = pl.Trainer(max_epochs=5, logger=False, enable_checkpointing=False)

# Run training
trainer.fit(model, dataloader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\neela\anaconda3\envs\torch_projects\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type    | Params | Mode 
----------------------------------------------
0 | model     | SegRNN  | 1.2 K  | train
1 | criterion | MSELoss | 0      | train
----------------------------------------------
1.2 K     Trainable params
0         Non-trainab

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.
